In [8]:
!pip install surprise

     |████████████████████████████████| 11.8MB 287kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617565 sha256=b3dbae6cd6fe5ed5f85a8e109cf441b29ca9cb8959f21f5dd92ba63f0c4a1f6f
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
#!pip install joblib

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import seaborn as sns
import joblib
from sklearn.model_selection import train_test_split
import gzip
import dask as dd
#import Recommender as Recommender
#import Evaluation as Evaluation

In [ ]:
# !wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Electronics.csv
# !wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Electronics.json.gz

In [ ]:
# def parse(path):
#   g = gzip.open(path, 'rb')
#   for l in g:
#     yield eval(l)

# def getDF(path):
#   i = 0
#   df = {}
#   for d in parse(path):
#     df[i] = d
#     i += 1
#   return pd.DataFrame.from_dict(df, orient='index')

# dfmeta = getDF('./meta_Electronics.json.gz')

In [ ]:
# dfmeta.head()

In [ ]:
# dfmeta.columns

In [ ]:
# dfreviews=pd.read_csv('./ratings_Electronics.csv', header=None,sep=',')

In [ ]:
# dfreviews.columns = ["reviewerID", "asin","overall","timestamp"]
# dfreviews.head()

In [ ]:
# product_ratings=pd.merge(dfreviews,dfmeta,on='asin',how='inner')

In [ ]:
# product_ratings.drop(['timestamp','description','tech1','fit','also_buy','brand','feature','rank','also_view','details','similar_item','date','price'], axis=1, inplace=True)

In [ ]:
# product_ratings.drop(['image'],axis=1,inplace=True)

In [3]:
product_ratings=pd.read_csv('./prod_ratings.csv')

In [4]:
product_ratings = product_ratings.rename(columns={'reviewerID': 'userID', 'asin': 'prod_ID','title': 'prod_name',
                                                 'overall': 'rating' })

In [5]:
product_ratings.head()

,userID,prod_ID,rating,imgurl,prod_name
0,A5JLAU2ARJ0BO,1400532655,1.0,http://ecx.images-amazon.com/images/I/51lBXZsM...,Barnes &amp; Noble Nook Color eReader
1,A1T1YSCDW0PD25,B00000DM9W,5.0,http://ecx.images-amazon.com/images/I/41N1GD82...,Sony ICF-S79V Weather Band Shower Radio (Disco...
2,A3CW0ZLUO5X2B1,B00000K135,5.0,http://ecx.images-amazon.com/images/I/71QD0N8R...,Brother M231 1/2-Inch Black on White Tape for ...
3,A22CW0ZHY3NJH8,B00000K135,5.0,http://ecx.images-amazon.com/images/I/71QD0N8R...,Brother M231 1/2-Inch Black on White Tape for ...
4,A231WM2Z2JL0U3,B00000K4KH,5.0,http://ecx.images-amazon.com/images/I/41SO%2B1...,Kensington 64068F MicroSaver Notebook Lock and...


In [9]:
from collections import defaultdict
from surprise import Dataset
from surprise import SVD
from surprise.model_selection import KFold
import pandas as pd
import numpy as np


## **How to compute precision@k and recall@k**
Here is an example where we compute Precision@k and Recall@k for each user:

Precision@k=|{Recommended items that are relevant}||{Recommended items}| Recall@k=|{Recommended items that are relevant}||{Relevant items}|

An item is considered relevant if its true rating rui is greater than a given threshold. An item is considered recommended if its estimated rating r^ui is greater than the threshold, and if it is among the k highest estimated ratings.

Note that in the edge cases where division by zero occurs, Precision@k and Recall@k values are undefined. As a convention, we set their values to 0 in such cases.

In [10]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

## **Use a custom dataset**

In [ ]:
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

# path to dataset file
file_path = os.path.expanduser('~/.surprise_data/ml-100k/ml-100k/u.data')

# As we're loading a custom dataset, we need to define a reader. In the
# movielens-100k dataset, each line has the following format:
# 'user item rating timestamp', separated by '\t' characters.
reader = Reader(line_format='user item rating timestamp', sep='\t')

data = Dataset.load_from_file(file_path, reader=reader)

# We can now use this dataset as we please, e.g. calling cross_validate
cross_validate(BaselineOnly(), data, verbose=True)

In [11]:
import pandas as pd

from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate


# Creation of the dataframe. Column names are irrelevant.
ratings_dict = {'itemID': [1, 1, 1, 2, 2],
                'userID': [9, 32, 2, 45, 'user_foo'],
                'rating': [3, 2, 4, 3, 1]}

df = pd.DataFrame(ratings_dict)

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(product_ratings[['userID','prod_ID','rating']], reader)

# We can now use this dataset as we please, e.g. calling cross_validate
cross_validate(NormalPredictor(), data, cv=2)

{'fit_time': (0.0037631988525390625, 0.0027723312377929688),
 'test_mae': array([0.86262402, 0.89169028]),
 'test_rmse': array([1.17593103, 1.20479803]),
 'test_time': (0.015383005142211914, 0.013213157653808594)}

In [12]:
# data = Dataset.load_builtin('ml-100k')
kf = KFold(n_splits=5)
algo = SVD()


In [13]:
fold=0
for trainset, testset in kf.split(data):
    fold+=1
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

    # Precision and recall can then be averaged over all users
    print('Fold: {}'.format(fold))
    print('precision value: {}'.format(sum(prec for prec in precisions.values()) / len(precisions)))
    print('precision value: {}'.format(sum(rec for rec in recalls.values()) / len(recalls)))

Fold: 1
precision value: 0.8244444444444444
precision value: 0.3370956200734501
Fold: 2
precision value: 0.7800000000000001
precision value: 0.3419982247295254
Fold: 3
precision value: 0.8222222222222224
precision value: 0.32170120436202315
Fold: 4
precision value: 0.8488888888888888
precision value: 0.33180042225807327
Fold: 5
precision value: 0.8355555555555556
precision value: 0.36322085915984537


## **Finding RMSE **

In [15]:
from surprise import Dataset
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import KFold


#data = Dataset.load_builtin('ml-100k')

algo = SVD()

trainset = data.build_full_trainset()
algo.fit(trainset)

testset = trainset.build_testset()
predictions = algo.test(testset)
# RMSE should be low as we are biased
accuracy.rmse(predictions, verbose=True)  # ~ 0.68 (which is low)

RMSE: 0.6232


0.6232182220042283

## **Tune algorithm parameters with GridSearchCV**

The cross_validate() function reports accuracy metric over a cross-validation procedure for a given set of parameters. If you want to know which parameter combination yields the best results, the GridSearchCV class comes to the rescue. Given a dict of parameters, this class exhaustively tries all the combinations of parameters and reports the best parameters for any accuracy measure (averaged over the different splits). It is heavily inspired from scikit-learn’s GridSearchCV.

Here is an example where we try different values for parameters n_epochs, lr_all and reg_all of the SVD algorithm.

In [16]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV

# Use movielens-100K
#data = Dataset.load_builtin('ml-100k')

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

0.8759788830949414
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [17]:
# We can now use the algorithm that yields the best rmse:
algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

In [18]:
results_df = pd.DataFrame.from_dict(gs.cv_results)

In [19]:
results_df

,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,std_test_mae,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_n_epochs,param_lr_all,param_reg_all
0,0.902635,0.874688,0.919938,0.899087,0.018643,8,0.692988,0.681431,0.697584,0.690668,0.006795,7,0.044452,0.000873,0.013644,0.003247,"{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}",5,0.002,0.4
1,0.901361,0.875540,0.919982,0.898961,0.018223,7,0.693621,0.682909,0.700567,0.692366,0.007263,8,0.046745,0.006727,0.008298,0.000255,"{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.6}",5,0.002,0.6
2,0.887724,0.861184,0.899683,0.882863,0.016088,3,0.679207,0.667521,0.679917,0.675548,0.005684,3,0.043065,0.000752,0.009448,0.001601,"{'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.4}",5,0.005,0.4
3,0.887425,0.862454,0.900771,0.883550,0.015881,4,0.680184,0.668456,0.682701,0.677114,0.006208,4,0.046484,0.002868,0.008155,0.000246,"{'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.6}",5,0.005,0.6
4,0.889550,0.863419,0.903793,0.885587,0.016719,5,0.680514,0.669056,0.682999,0.677523,0.006072,5,0.083333,0.005714,0.011444,0.004599,"{'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.4}",10,0.002,0.4
5,0.890317,0.866485,0.906054,0.887619,0.016266,6,0.681410,0.672569,0.688985,0.680988,0.006708,6,0.075386,0.000675,0.009189,0.001974,"{'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.6}",10,0.002,0.6
6,0.882034,0.854553,0.891349,0.875979,0.015620,1,0.673390,0.659674,0.673385,0.668816,0.006465,1,0.081686,0.007443,0.008086,0.000263,"{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}",10,0.005,0.4
7,0.882507,0.858118,0.894659,0.878428,0.015194,2,0.674654,0.662789,0.677500,0.671647,0.006371,2,0.074313,0.000995,0.007931,0.000241,"{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.6}",10,0.005,0.6


Comparison of two algorithms
## **We will see in this notebook how we can compare the prediction accuracy of two algorithms.**

In [20]:
from __future__ import (absolute_import, division, print_function,             
                        unicode_literals)                                      
import pickle
import os

import pandas as pd

from surprise import SVD
from surprise import KNNBasic
from surprise import Dataset                                                     
from surprise import Reader                                                      
from surprise.model_selection import PredefinedKFold
from surprise import dump
from surprise.accuracy import rmse

In [21]:
# We will train and test on the u1.base and u1.test files of the movielens-100k dataset.
# if you haven't already, you need to download the movielens-100k dataset
# You can do it manually, or by running:

# Dataset.load_builtin('ml-100k')

# Now, let's load the dataset
# train_file = os.path.expanduser('~') + '/.surprise_data/ml-100k/ml-100k/u1.base'
# test_file = os.path.expanduser('~') + '/.surprise_data/ml-100k/ml-100k/u1.test'
# data = Dataset.load_from_folds([(train_file, test_file)], Reader('ml-100k'))

# define a cross-validation iterator
pkf = KFold(n_splits=3)

                
# We'll use the well-known SVD algorithm and a basic nearest neighbors approach.
algo_svd = SVD()                                                       
algo_knn = KNNBasic()

for trainset, testset in pkf.split(data): 
    algo_svd.fit(trainset)                             
    predictions_svd = algo_svd.test(testset)
    
    algo_knn.fit(trainset)
    predictions_knn = algo_knn.test(testset)
    
    rmse(predictions_svd)
    rmse(predictions_knn)                                                                           
    
    dump.dump('./dump_SVD', predictions_svd, algo_svd)
    dump.dump('./dump_KNN', predictions_knn, algo_knn)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8928
RMSE: 0.9792
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8556
RMSE: 1.0244
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8692
RMSE: 1.0296


In [22]:
# The dumps have been saved and we can now use them whenever we want.

predictions_svd, algo_svd = dump.load('./dump_SVD')
predictions_knn, algo_knn = dump.load('./dump_KNN')

df_svd = pd.DataFrame(predictions_svd, columns=['uid', 'iid', 'rui', 'est', 'details'])    
df_knn = pd.DataFrame(predictions_knn, columns=['uid', 'iid', 'rui', 'est', 'details'])    

df_svd['err'] = abs(df_svd.est - df_svd.rui)
df_knn['err'] = abs(df_knn.est - df_knn.rui)

In [23]:
df_svd.head()

,uid,iid,rui,est,details,err
0,A2AY4YUOX2N1BQ,B000VX6XL6,5.0,4.525171,{'was_impossible': False},0.474829
1,A20DZX38KRBIT8,B000JLKIHA,5.0,3.305755,{'was_impossible': False},1.694245
2,ARBKYIVNYWK3C,B00CYX54C0,4.0,4.513844,{'was_impossible': False},0.513844
3,A4WEZJOIZIV4U,B0046RE21G,5.0,4.573964,{'was_impossible': False},0.426036
4,A3AYSYSLHU26U9,B000MFG44Y,3.0,4.478592,{'was_impossible': False},1.478592


In [24]:
df_knn.head()

,uid,iid,rui,est,details,err
0,A2AY4YUOX2N1BQ,B000VX6XL6,5.0,4.613260,"{'actual_k': 6, 'was_impossible': False}",0.386740
1,A20DZX38KRBIT8,B000JLKIHA,5.0,4.408326,"{'was_impossible': True, 'reason': 'User and/o...",0.591674
2,ARBKYIVNYWK3C,B00CYX54C0,4.0,4.408326,"{'was_impossible': True, 'reason': 'User and/o...",0.408326
3,A4WEZJOIZIV4U,B0046RE21G,5.0,4.408326,"{'was_impossible': True, 'reason': 'User and/o...",0.591674
4,A3AYSYSLHU26U9,B000MFG44Y,3.0,4.408326,"{'was_impossible': True, 'reason': 'User and/o...",1.408326


In [27]:
# How different are the predictions from both algorithms ?
# Let's count the number of predictions for each rating value

import matplotlib.pyplot as plt
import matplotlib
%matplotlib notebook
matplotlib.style.use('ggplot')

figure, (ax1, ax2) = plt.subplots(1, 2)

df_svd.est.plot(kind='hist', title='SVD', ax=ax1)
df_knn.est.plot(kind='hist', title='KNN', ax=ax2)
# As expected, one of the drawbacks of the NN algorithms is that their predictions are often
# quite concentrated around the mean. The SVD algorithm seems more confortable predicting extreme rating values.

<IPython.core.display.Javascript object>

In [26]:
# gives the best predictions on average?

def get_Iu(uid):
    """Return the number of items rated by given user
    
    Args:
        uid: The raw id of the user.
    Returns:
        The number of items rated by the user.
    """
    
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError:  # user was not part of the trainset
        return 0
    
df_knn['Iu'] = df_knn.uid.apply(get_Iu)
df_svd['Iu'] = df_svd.uid.apply(get_Iu)

df_knn[df_knn.Iu < 10].err.mean(), df_svd[df_svd.Iu < 10].err.mean()

(0.5183347350714886, 0.5377405184112355)

## **Automatic** cross-validation

In [29]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate


# Load the movielens-100k dataset (download it if needed),
#data = Dataset.load_builtin('ml-100k')

# We'll use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8493  0.8344  0.8797  0.8704  0.9274  0.8722  0.0318  
MAE (testset)     0.6470  0.6373  0.6574  0.6567  0.6627  0.6522  0.0090  
Fit time          0.18    0.17    0.18    0.18    0.19    0.18    0.00    
Test time         0.01    0.01    0.01    0.01    0.01    0.01    0.00    


{'fit_time': (0.1750338077545166,
  0.17369961738586426,
  0.18011713027954102,
  0.1834094524383545,
  0.18511629104614258),
 'test_mae': array([0.64697036, 0.63725173, 0.65741261, 0.65667239, 0.66268192]),
 'test_rmse': array([0.84928524, 0.83444725, 0.87968797, 0.87035522, 0.92735562]),
 'test_time': (0.005154609680175781,
  0.005031108856201172,
  0.0050733089447021484,
  0.0051593780517578125,
  0.00543522834777832)}